In [ ]:
import requests
from bs4 import BeautifulSoup
import wget
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [ ]:
path_to_directory = 'PATH_TO_YOUR_FOLDER'

In [ ]:
def download_podcasts(soup, title):
  i = 0
  time_all = [] #Dates of realeases of the podcast episodes
  ID_all = [] #ID of the podcast episodes
  description_all = [] #Description of each podcast episode
  length_all = [] #Duration of each podcast episode
  link_all = [] #Link to each podcast episode
  name_all = [] #Names of the podcast episodes
  for podcast in soup.find_all('a', {'role': 'listitem'}):
    time = podcast.find('div', {'class': 'OTz6ee'}).text
    #Downloading all episodes released in December 2022
    if (time.startswith('Dec')) and (time.endswith('2022')):
      i += 1
      time_all.append(time) #Date of release of the episode

      link = podcast.find('div', {'jsname': 'fvi9Ef'})['jsdata'].split(';')[1]
      link_all.append(link) #Link for the episode

      name = podcast.find('div', {'class': 'e3ZUqe'}).text
      name_all.append(name) #Name of the episode

      print(i, ':', time)
      filename = wget.download(link, out=path_to_directory+title) #Downloading the episode audio file
      os.rename(filename, path_to_directory+title+'/audio'+str(i)+'.mp3')
      ID_all.append('audio'+str(i)) #ID for the episode

      #Looking to see if the description of the episode is present
      try:
        description = podcast.find('div', {'class': 'LrApYe'}).text
      except:
        description = 'None'

      if description is None:
        description = 'None'
      description_all.append(description)

      length = podcast.find('span', {'class': 'gUJ0Wc'}).text
      length_all.append(length) #Duration of the episode

  #Saving all the information relating to the podcast episode as a pandas dataframe
  df = pd.DataFrame(list(zip([title]*len(ID_all), ID_all, name_all,
                             time_all,
                             description_all,
                             length_all,
                             link_all)),
                    columns = ['Show', 'ID', 'Episode', 'Time',
                               'Description', 'Length', 'Link'])
  return df

In [ ]:
#URLs of the podcasts: 'The Stack Overflow Podcast' and 'Myths and Legends'
URLs = ['LINKS_TO_PODCASTS']

info_df_list = []

for url in URLs:
  #Using BeautifulSoup to scrape and parse HTML content from the web pages of the two podcasts using lxml parser
  soup = BeautifulSoup(requests.get(url).text, 'lxml')
  title = soup.find('div', {'class': 'ZfMIwb'}).text #To retrieve the title of the podcast
  os.mkdir(path_to_directory+title)
  df = download_podcasts(soup, title) #Calling the previously defined function to download the relevant episodes and return their data
  info_df_list.append(df)

info_all_podcasts = pd.concat(info_df_list)

In [ ]:
info_all_podcasts

In [ ]:
#Saving the dataframe as a csv file for future use
info_all_podcasts.to_csv(path_to_directory+'podcast.csv')